In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

import numpy as np
import tensorflow as tf
import time

Up until now, we've been running our code in "[eager execution](https://www.tensorflow.org/guide/eager)" mode, which is enabled by default. In this mode, the flow of code execution happens in the order we're accustomed to, and we can add breakpoints and inspect the values of our tensors and variables as usual.

In contrast, when in "[graph execution](https://www.tensorflow.org/guide/intro_to_graphs)" mode, the code execution flows a bit differently: during the first pass through the code, a computation graph is created containing information about the operations and tensors in that code. Then in subsequent passes, the graph is used instead of the Python code. One consequence of this flow is that our code is not debuggable in the usual manner. We gain two major advantages though:
- The graph can be deployed to environments that don't have Python, such as embedded devices. 
- The graph can take advantage of several performance optimizations, such as running parts of the code in parallel.

In order to get the best of both worlds, we use eager execution mode during the development phase, and then switch to graph execution mode once we're done debugging the model. To switch from eager to graph execution, we can add the `@tf.function` decorator to the function containing our model operations.

Let's look at the training code again, but this time with the `@tf.function` decorator applied to the `fit_one_batch` function, which is where we have all the model operations.

In [2]:
!wget -Nq https://raw.githubusercontent.com/MicrosoftDocs/tensorflow-learning-path/main/intro-tf/tintro.py
from tintro import *

In [3]:
@tf.function
def fit_one_batch(X: tf.Tensor, y: tf.Tensor, model: tf.keras.Model, loss_fn: tf.keras.losses.Loss, 
optimizer: tf.keras.optimizers.Optimizer) -> Tuple[tf.Tensor, tf.Tensor]:
  with tf.GradientTape() as tape:
    y_prime = model(X, training=True)
    loss = loss_fn(y, y_prime)

  grads = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  return (y_prime, loss)


def fit(dataset: tf.data.Dataset, model: tf.keras.Model, loss_fn: tf.keras.losses.Loss, 
optimizer: tf.optimizers.Optimizer) -> None:
  batch_count = len(dataset)
  loss_sum = 0
  correct_item_count = 0
  current_item_count = 0
  print_every = 100

  for batch_index, (X, y) in enumerate(dataset):
    (y_prime, loss) = fit_one_batch(X, y, model, loss_fn, optimizer)

    y = tf.cast(y, tf.int64)
    correct_item_count += (tf.math.argmax(y_prime, axis=1) == y).numpy().sum()

    batch_loss = loss.numpy()
    loss_sum += batch_loss
    current_item_count += len(X)

    if ((batch_index + 1) % print_every == 0) or ((batch_index + 1) == batch_count):
      batch_accuracy = correct_item_count / current_item_count * 100
      print(f'[Batch {batch_index + 1:>3d} - {current_item_count:>5d} items] accuracy: {batch_accuracy:>0.1f}%, loss: {batch_loss:>7f}')


learning_rate = 0.1
batch_size = 64
epochs = 5

(train_dataset, test_dataset) = get_data(batch_size)

model = NeuralNetwork()

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.optimizers.SGD(learning_rate)

print('\nFitting:')
t_begin = time.time()
for epoch in range(epochs):
  print(f'\nEpoch {epoch + 1}\n-------------------------------')
  fit(train_dataset, model, loss_fn, optimizer)
t_elapsed = time.time() - t_begin
print(f'\nTime per epoch: {t_elapsed / epochs :>.3f} sec' )


Fitting:

Epoch 1
-------------------------------
[Batch 100 -  6400 items] accuracy: 60.1%, loss: 0.897071
[Batch 200 - 12800 items] accuracy: 67.3%, loss: 0.578438
[Batch 300 - 19200 items] accuracy: 70.9%, loss: 0.557362
[Batch 400 - 25600 items] accuracy: 73.0%, loss: 0.760332
[Batch 500 - 32000 items] accuracy: 74.4%, loss: 0.827212
[Batch 600 - 38400 items] accuracy: 75.4%, loss: 0.288966
[Batch 700 - 44768 items] accuracy: 76.5%, loss: 0.672835
[Batch 800 - 51168 items] accuracy: 77.1%, loss: 0.716132
[Batch 900 - 57568 items] accuracy: 77.6%, loss: 0.504184
[Batch 938 - 60000 items] accuracy: 77.8%, loss: 0.445008

Epoch 2
-------------------------------
[Batch 100 -  6400 items] accuracy: 82.2%, loss: 0.480275
[Batch 200 - 12800 items] accuracy: 83.0%, loss: 0.510372
[Batch 300 - 19200 items] accuracy: 83.2%, loss: 0.764944
[Batch 400 - 25600 items] accuracy: 83.2%, loss: 0.617230
[Batch 500 - 32000 items] accuracy: 83.2%, loss: 0.440339
[Batch 600 - 38400 items] accuracy: 83

Notice that we also add a timer, and print the time it takes to train. You can comment and uncomment the `@tf.function` decorator, and notice the difference between the elapsed times. On my machine, eager execution takes more than twice the amount of time to train, compared to graph execution.

Now that we've trained our model, we're ready to test it, which we can do by running a single pass forward through the network. The function `evaluate_one_batch` contains the code that does this: we simply need to call the `model` to get a prediction, followed by the loss function `loss_fn` to get a score for how the predicted labels `y_prime` compare to the actual labels `y`. Notice that we don't add a `tf.GradientTape()` this time &mdash; that's because, since we don't do a backward pass during testing, we don't need to calculate derivatives for gradient descent. Notice also that we added a `@tf.function` decorator once we were done with development and debugging, to get a performance boost.  

In [4]:
@tf.function
def evaluate_one_batch(X: tf.Tensor, y: tf.Tensor, model: tf.keras.Model, 
loss_fn: tf.keras.losses.Loss) -> Tuple[tf.Tensor, tf.Tensor]:
  y_prime = model(X, training=False)
  loss = loss_fn(y, y_prime)

  return (y_prime, loss)

The `evaluate` function calls the `evaluate_one_batch` function for the entire dataset, once per mini-batch. The important code in the function below is just the `for` loop and the call to `evaluate_one_batch` within it &mdash; the rest is just boilerplate code to print progress during execution.

In [5]:
def evaluate(dataset: tf.data.Dataset, model: tf.keras.Model, 
loss_fn: tf.keras.losses.Loss) -> Tuple[float, float]:
  batch_count = len(dataset)
  loss_sum = 0
  correct_item_count = 0
  current_item_count = 0

  for (X, y) in dataset:
    (y_prime, loss) = evaluate_one_batch(X, y, model, loss_fn)

    correct_item_count += (tf.math.argmax(y_prime, axis=1).numpy() == y.numpy()).sum()
    loss_sum += loss.numpy()
    current_item_count += len(X)

  average_loss = loss_sum / batch_count
  accuracy = correct_item_count / current_item_count
  return (average_loss, accuracy)

And finally, we print the test loss and accuracy, and save the learned model parameters.

In [6]:
print('\nEvaluating:')
(test_loss, test_accuracy) = evaluate(test_dataset, model, loss_fn)
print(f'Test accuracy: {test_accuracy * 100:>0.1f}%, test loss: {test_loss:>8f}')

model.save_weights('outputs/weights')


Evaluating:
Test accuracy: 84.9%, test loss: 0.424060


The training loss and accuracy should be similar to the values we obtained with the Keras code. 